In [129]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager


# use GeckoDriver manager to access firefox page via gecko
executable = FirefoxService(GeckoDriverManager().install())

driver = webdriver.Firefox(service=executable)

[WDM] - Downloading: 16.9kB [00:00, 8.38MB/s]                   


In [130]:
url_base = "https://www.indeed.com/jobs?q="
pg = 0

pos = 'software engineer'
position = '+'.join(pos.split())

url_search = url_base + position

In [186]:
page = driver.get(url_search)   
driver.implicitly_wait(10)

# pull job cards, excluding final blank card
job_cards = driver.find_elements(By.CSS_SELECTOR,".jobsearch-ResultsList > li")[:-1]

# remove spacer cards
job_cards.pop(5)
job_cards.pop(10)

<selenium.webdriver.remote.webelement.WebElement (session="62666712-0603-4110-a0d0-fcfcfb3eacdc", element="25056a7d-c240-4dbf-81ab-50b309edfbf5")>

In [187]:
criteria = ['job_title','company','location','salary','employment_type','description']
df = pd.DataFrame(index = range(len(job_cards)),columns=criteria)
df

,job_title,company,location,salary,employment_type,description
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
all_titles = []
all_companies = []
all_loc = []
all_salary = []
all_type = []
all_text = []

for idx,job in enumerate(job_cards):
#     if idx == 5 or idx == 11: continue
#     print(idx)
    
    soup = BeautifulSoup(job.get_attribute('innerHTML'), 'html.parser')

    # get html from root node
    soup_children = list(soup.children)
    html = soup_children[0]    

    # Get head from html 
    html_children = list(html.children)
    head = html_children[0]
    
    # get job card stats
    job_title = head.find('h2').text
    company = head.find("span", {"class":"companyName"}).text
    location = head.find("div",{"class":"companyLocation"}).text
    header_meta = head.find_all("div",{"class":"metadata"})
    
    if header_meta:
        chars = header_meta[0].text
        if any(c.isdigit() for c in chars):
            salary = chars
            
        else:
            salary = 'n/a'
            job_type = chars
        
        try:
            job_type = header_meta[1].text
        
        except:
            job_type = 'n/a'
    
    all_titles.append(job_title)
    all_companies.append(company)
    all_loc.append(location)
    all_salary.append(salary)
    all_type.append(job_type)
    
df['job_title'] = all_titles
df['company'] = all_companies
df['location'] = all_loc
df['salary'] = all_salary
df['employment_type'] = all_type
# df['description'] = all_text

df

,job_title,company,location,salary,employment_type,description
0,"Java Developer - Jersey City, NJ",NTT DATA,"Jersey City, NJ 07302 (Downtown area)",Estimated $66.3K - $83.9K a year,n/a,NaN
1,Software Engineer Lead - Digital,Best Version Media,"Hybrid remote in Brookfield, WI 53045","From $100,000 a year",Full-time,NaN
2,Software Engineer I - Full Stack,Indeed,"Remote in Austin, TX","$90,000 - $130,000 a year",Full-time,NaN
3,Sr. Full Stack Developer - Ruby and Scala,FLEETCOR,Remote in Arizona,n/a,Holidays,NaN
4,Java Developer (ReEMERGE: Return To Work Program),DTCC,"Jersey City, NJ 07310 (The Waterfront area)",Estimated $93.4K - $118K a year,Internship,NaN
5,Software Engineer,BringFido,Remote,Estimated $89.1K - $113K a year,Full-time,NaN
6,Software Engineer I,"MISTRAS Group, Inc.",Remote,"$65,000 - $80,000 a year",Full-time,NaN
7,Backend Software Engineer (All Skill Levels),Wolfjaw Studios,Remote,"$60,000 - $100,000 a year",Full-time,NaN
8,Software Engineer I - Radar (C++),Raytheon Missiles & Defense,"Marlborough, MA 01752",n/a,n/a,NaN
9,Software Engineer I (React),EON,"Remote in New York, NY","$85,000 - $120,000 a year",n/a,NaN
